In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q groq qdrant-client sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 18.6 MB/s eta 0:00:00


In [ ]:
import os
import uuid
import re
import json
import concurrent.futures
import io
import time
import smtplib
import threading
from datetime import datetime
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer

# --- OPTIONAL VOICE DEPENDENCIES ---
try:
    from gtts import gTTS
    import pygame
    try:
        pygame.mixer.pre_init(44100, -16, 2, 2048)
        pygame.mixer.init()
        TTS_SUPPORT = True
    except (pygame.error, Exception):
        TTS_SUPPORT = False
except ImportError:
    TTS_SUPPORT = False

# --- CONSTANTS ---
STOPWORDS = {
    "about", "after", "again", "all", "almost", "also", "always", "and", 
    "because", "before", "can", "could", "does", "down", "for", "from", 
    "have", "having", "here", "how", "into", "just", "know", "like", 
    "more", "most", "much", "next", "not", "now", "only", "out", "over", 
    "people", "should", "some", "still", "such", "than", "that", "the", 
    "then", "there", "these", "this", "time", "today", "very", "want", 
    "was", "what", "when", "where", "which", "who", "will", "with", 
    "would", "year", "your", "it's", "its", "user", "feels" 
}

# --- 1. EMAIL SYSTEM ---
class EmailDispatcher:
    def __init__(self, sender_email=None, sender_password=None, recipient_email=None):
        self.sender_email = sender_email
        self.sender_password = sender_password
        self.recipient_email = recipient_email
        self.mock_mode = not (sender_email and sender_password and recipient_email)
        
        if self.mock_mode:
            print("   [System] Email credentials missing. Running in MOCK MODE.")

    def send_email(self, subject, body_text, body_html=None):
        print(f"\n   [📧 EMAIL LOG] Dispatching: '{subject}'")
        if self.mock_mode:
            print("   " + "="*50)
            print(f"   [SIMULATION] To: {self.recipient_email or 'User'}\n   [SIMULATION] Body: {body_text}")
            print("   " + "="*50)
            return True

        try:
            msg = MIMEMultipart("alternative")
            msg["Subject"] = subject
            msg["From"] = self.sender_email
            msg["To"] = self.recipient_email
            msg.attach(MIMEText(body_text, "plain"))
            if body_html:
                msg.attach(MIMEText(body_html, "html"))

            with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
                server.login(self.sender_email, self.sender_password)
                server.send_message(msg)
            return True
        except Exception as e:
            print(f"   [❌ Error] Email failed: {e}")
            return False

# --- 2. SENTINEL (Safety Monitor) ---
class Sentinel:
    def __init__(self, bot_instance, mailer_instance):
        self.client = bot_instance.client
        self.memory = bot_instance.memory
        self.mailer = mailer_instance
        self.model = "llama-3.1-8b-instant"

    def _analyze_trends(self, memories):
        mem_text = "\n".join([f"- {m}" for m in memories])
        prompt = f"""You are 'The Sentinel', a safety monitor. Analyze these memories for triggers:
        1. HIGH DISTRESS: Is the user spiraling?
        2. EVENTS: Recent specific events to check on.
        USER MEMORIES: {mem_text}
        Output purely JSON: {{"status": "ALERT/STABLE", "reason": "...", "action_type": "CHECK_IN/NONE", "topic": "..."}}"""
        
        try:
            resp = self.client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=self.model, temperature=0.1, response_format={"type": "json_object"}
            )
            return json.loads(resp.choices[0].message.content)
        except:
            return {"status": "STABLE", "action_type": "NONE"}

    def _draft_notification(self, reason, topic, username):
        prompt = f"""Write a 2-sentence warm, gentle check-in email. 
        Context: The user is feeling {reason} regarding {topic}.
        
        STRICT RULES:
        1. Address the user as '{username}'.
        2. DO NOT include introductory text like "Here is your email" or "Subject:".
        3. DO NOT use placeholders like [Your Name] or [User Name].
        4. Sign off as 'Vigil AI'.
        5. Output ONLY the body of the email."""
        
        try:
            resp = self.client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=self.model, temperature=0.7
            )
            return resp.choices[0].message.content.strip()
        except:
            return f"Hi {username}, just thinking of you and checking in. Vigil AI is here if you need to talk."

    def _dispatch_with_delay(self, subject, llm_message, html_body):
        """Internal function to handle the sleep and send, intended for threading."""
        # Delay set to 3600 seconds (1 hour)
        time.sleep(3600)
        self.mailer.send_email(subject, llm_message, html_body)

    def run_check(self, username):
        try:
            raw_data = self.memory.client.scroll(collection_name=self.memory.collection_name, limit=10)[0]
            recent_memories = [p.payload['text'] for p in raw_data]
            if not recent_memories: return

            analysis = self._analyze_trends(recent_memories)
            if analysis.get('status') == "ALERT" or analysis.get('action_type') == "CHECK_IN":
                llm_message = self._draft_notification(analysis.get('reason'), analysis.get('topic', 'mood'), username)
                subject = f"Checking in: Regarding {analysis.get('topic', 'how you are doing')}"
                html_body = f"""
                <html>
                <body style='font-family: Arial, sans-serif; color: #333;'>
                    <div style='max-width: 500px; padding: 20px; border: 1px solid #eee; border-radius: 10px;'>
                        <h2 style='color: #4A90E2;'>Vigil AI Care Team</h2>
                        <p style='font-size: 16px; line-height: 1.5;'>{llm_message}</p>
                    </div>
                </body>
                </html>
                """
                # Start delayed dispatch in a separate thread to keep chat responsive
                threading.Thread(
                    target=self._dispatch_with_delay, 
                    args=(subject, llm_message, html_body),
                    daemon=True
                ).start()
        except Exception as e:
            print(f"   [Sentinel Error] {e}")

# --- 3. MEMORY MANAGER ---
class MemoryManager:
    def __init__(self):
        print("   [System] Initializing Memory Core...")
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.client = QdrantClient(location=":memory:")
        self.collection_name = "therapy_memories"
        self.client.recreate_collection(
            collection_name=self.collection_name,
            vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
        )

    def _tokenize(self, text):
        clean_text = re.sub(r'[^\w\s]', '', text.lower())
        words = {w for w in clean_text.split() if w not in STOPWORDS and len(w) > 3}
        return words

    def add_memory(self, text):
        # Removed the console logging for database writes as requested
        vector = self.encoder.encode(text).tolist()
        payload = {"text": text, "timestamp": datetime.now().isoformat()}
        self.client.upsert(
            collection_name=self.collection_name,
            points=[models.PointStruct(id=str(uuid.uuid4()), vector=vector, payload=payload)]
        )

    def search_memories(self, query_text, limit=3, score_threshold=0.35):
        query_vector = self.encoder.encode(query_text).tolist()
        hits = []
        found_texts = set()
        try:
            results = self.client.search(collection_name=self.collection_name, query_vector=query_vector, limit=limit, score_threshold=score_threshold)
            for hit in results:
                hits.append(hit.payload['text'])
                found_texts.add(hit.payload['text'])
        except Exception: pass
        if len(hits) < limit:
            try:
                query_keywords = self._tokenize(query_text)
                if query_keywords:
                    all_points = self.client.scroll(collection_name=self.collection_name, limit=100)[0]
                    for point in all_points:
                        text = point.payload['text']
                        if text in found_texts: continue
                        if query_keywords.intersection(self._tokenize(text)):
                            hits.append(text); found_texts.add(text)
                            if len(hits) >= limit: break
            except: pass
        return hits[:limit]

# --- 4. STT & TTS AGENTS ---
class STTAgent:
    def __init__(self, client): self.client = client
    def transcribe_file(self, file_path):
        if not os.path.exists(file_path): return f"Error: File '{file_path}' not found."
        try:
            with open(file_path, "rb") as file:
                transcription = self.client.audio.transcriptions.create(file=(os.path.basename(file_path), file), model="whisper-large-v3", response_format="text")
            return str(transcription).strip()
        except Exception as e: return f"STT Agent Error: {e}"

class TTSAgent:
    def __init__(self):
        try:
            from gtts import gTTS
            self.gtts_available = True
        except ImportError: self.gtts_available = False
        self.can_play = TTS_SUPPORT
    def speak(self, text):
        if not self.gtts_available: return
        try:
            tts = gTTS(text=text, lang='en')
            temp_file = "last_reply.mp3"
            tts.save(temp_file)
            if self.can_play:
                pygame.mixer.music.load(temp_file); pygame.mixer.music.play()
                while pygame.mixer.music.get_busy(): time.sleep(0.1)
                pygame.mixer.music.unload()
        except: pass

# --- 5. SPECIALIST AGENTS ---
class SpecialistAgents:
    def __init__(self, client):
        self.client = client
        self.model = "llama-3.1-8b-instant"
    def _call_llm(self, sys, usr, temp=0.6):
        try:
            resp = self.client.chat.completions.create(messages=[{"role": "system", "content": sys}, {"role": "user", "content": usr}], model=self.model, temperature=temp, max_tokens=1000)
            return resp.choices[0].message.content
        except Exception as e: return f"Error: {e}"
    def run_validator(self, text):
        return self._call_llm("You are 'The Validator'. Focus on empathy and warmth. Keep it under 2 sentences.", text)
    def run_challenger(self, text):
        return self._call_llm("You are 'The Challenger'. Identify CBT distortions. Be analytical. Keep it under 2 sentences.", text)
    def run_summarizer(self, text):
        prompt = """Summarize this text into a concise memory key. Output ONLY the memory key string. STRICTLY FORBIDDEN: Do not include "I'm sorry", "Here is your key", "User feels", or conversational filler. PRESERVE proper nouns and specific events. Format: 'Sadness about [specific event] on [date/time]'."""
        return self._call_llm(prompt, text, temp=0.1)
    def run_synthesizer(self, original, validation, challenge, history, has_history, state_instr):
        prompt = f"You are 'Vigil' (formerly Anamnesis), a Lead Therapist.\n\n### TEAM REPORTS:\n1. VALIDATOR: {validation}\n2. CHALLENGER: {challenge}\n3. HISTORIAN: {history}\n\n### DYNAMIC INSTRUCTIONS:\n{state_instr}\n\n### FINAL GOAL:\nCraft a natural, seamless response. No jargon. No JSON. Reference history if {has_history}."
        return self._call_llm(prompt, original)

# --- 6. COUNCIL ORCHESTRATOR ---
class CouncilOrchestrator:
    def __init__(self):
        # Hardcoded Groq API Key as requested
        self.api_key = "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"
        os.environ["GROQ_API_KEY"] = self.api_key
        self.client = Groq(api_key=self.api_key)
        self.memory = MemoryManager()
        self.stt_agent = STTAgent(self.client); self.tts_agent = TTSAgent(); self.agents = SpecialistAgents(self.client)
        self.turn_count = 0; self.last_input_length = 0
    def _is_prime(self, n):
        if n < 2: return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0: return False
        return True
    def consult(self, user_input):
        self.turn_count += 1
        words = user_input.split(); input_len = len(words)
        state_instr = ""
        if input_len > 25:
            q = "Only ask a question if turn count is a prime number." if self._is_prime(self.turn_count) else "Stay in quiet active listening mode."
            state_instr = f"USER IS VENTING. {q} Focus on validation."
        elif input_len < 6 and self.last_input_length > 20: state_instr = "USER IS COLLAPSING. Step in now: Summarize their session, use a brief analogy, and provide a path forward."
        elif self.turn_count < 3: state_instr = "PROBING PHASE. Be brief. Ask one gentle question."
        else: state_instr = "Normal dialogue. Maintain flow. Use Challenger logic gently."
        self.last_input_length = input_len
        with concurrent.futures.ThreadPoolExecutor() as exec:
            f_val = exec.submit(self.agents.run_validator, user_input)
            f_chal = exec.submit(self.agents.run_challenger, user_input)
            f_hist = exec.submit(self.memory.search_memories, user_input)
            f_mem = exec.submit(self.agents.run_summarizer, user_input)
            v, c, h, m = f_val.result(), f_chal.result(), f_hist.result(), f_mem.result()
        history_text = "\n".join([f"- {i}" for i in h]) if h else "No context."
        final = self.agents.run_synthesizer(user_input, v, c, history_text, len(h)>0, state_instr)
        self.memory.add_memory(m)
        return final

# --- 7. CONTINUOUS CHAT ---
def start_chat():
    print("\n" + "="*60 + "\n      VIGIL AI: THE SENTINEL COUNCIL\n" + "="*60 + "\n")
    
    # Persistent User Logic
    username = input("Username: ").strip().lower()
    
    # Path for Kaggle or Local Environment
    user_db_file = "vigil_users.json"
    if os.path.exists("/kaggle/working"):
        user_db_file = os.path.join("/kaggle/working", user_db_file)
    
    users = {}
    if os.path.exists(user_db_file):
        try:
            with open(user_db_file, 'r') as f:
                users = json.load(f)
        except:
            users = {}
    
    email = users.get(username)
    if not email:
        email = input(f"New user '{username}' detected. Please provide your email for safety alerts: ").strip()
        users[username] = email
        with open(user_db_file, 'w') as f:
            json.dump(users, f)
    else:
        print(f"Welcome back, {username} ({email})")

    bot = CouncilOrchestrator()
    mailer = EmailDispatcher("ishansingh.iwg@gmail.com", "fkpy jpoe bkxv dvyb", email)
    sentinel = Sentinel(bot, mailer)
    voice_enabled = bot.tts_agent.gtts_available 

    print("\nVIGIL: I'm here. Whatever you're carrying, we can look at it together.\n")

    while True:
        try:
            user_input = input("YOU: ").strip()
            if user_input.lower() in ['exit', 'quit']: break
            if user_input.lower() == 'mute':
                voice_enabled = not voice_enabled
                print(f"   [System] Voice: {'ON' if voice_enabled else 'OFF'}")
                continue
            if user_input.lower() == 'v':
                path = input("Audio path: ").strip()
                user_input = bot.stt_agent.transcribe_file(path)
                if "Error" in user_input: print(user_input); continue
                print(f"(Transcribed): {user_input}")
            if not user_input: continue

            response = bot.consult(user_input)
            print(f"\nVIGIL: {response}\n")
            if voice_enabled: bot.tts_agent.speak(response)

            if bot.turn_count % 3 == 0:
                sentinel.run_check(username)
                
        except KeyboardInterrupt: break

if __name__ == "__main__":
    start_chat()

2026-01-22 18:10:29.409129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769105429.649015      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769105429.725317      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769105430.282818      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769105430.282862      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769105430.282865      55 computation_placer.cc:177] computation placer alr


      VIGIL AI: THE SENTINEL COUNCIL



Username:  daksh
New user 'daksh' detected. Please provide your email for safety alerts:  dadeech@mgmail.com


   [System] Initializing Memory Core...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_55/1058259498.py:167: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  self.client.recreate_collection(



VIGIL: I'm here. Whatever you're carrying, we can look at it together.



YOU:  im not feeling well at all



VIGIL: I'm so sorry to hear that you're not feeling well. Can you tell me a bit more about what's going on - is it physical or emotional, or a bit of both?

